##**Data Prep - Predição de Churn para Empresa de Telecomunicações**



Neste notebook, será realizada a etapa de preparação de dados (Data Preparation) a partir de um conjunto disponibilizado pela IBM, que reúne informações de mais de 7.000 clientes de uma empresa de telecom. O objetivo é estruturar e tratar esses dados para que possam ser utilizados no desenvolvimento de um modelo preditivo capaz de estimar a probabilidade de um cliente cancelar seus serviços.


**Resumo da Preparação dos Dados**

O processo de preparação dos dados para previsão de churn na empresa de telecomunicações seguiu várias etapas estruturadas para garantir qualidade, consistência e reprodutibilidade:

1. Conversão da variável alvo

   - A coluna Churn foi transformada de categórica (“Yes”/“No”) para binária (1/0), permitindo análises estatísticas e aplicação de algoritmos de machine learning.

2. Divisão em treino e teste (Holdout 70/30)

    - O dataset foi dividido em 70% para treino e 30% para teste, assegurando avaliação em dados não vistos e reduzindo risco de overfitting.

3. Tratamento de valores nulos

    - Nas variáveis categóricas, os nulos foram preenchidos pela moda.

    - As modas utilizadas foram registradas e salvas em arquivo .pkl para reaplicação no conjunto de teste.

    - Variáveis com alto percentual de nulos (≥70%) foram descartadas.

4. Exclusão de variáveis irrelevantes

    - Colunas sem valor preditivo, como Unnamed: 0, customerID e a própria variável alvo Churn, foram removidas antes do pré-processamento.

5. Tratamento de variáveis categóricas

    - Alta cardinalidade: transformadas por Label Encoding. Os encoders foram armazenados em .pkl para garantir consistência entre treino e teste.

    - Baixa cardinalidade: transformadas por One-Hot Encoding. O encoder e as colunas correspondentes também foram salvos em .pkl.

6. Padronização de variáveis numéricas

    - Aplicado o StandardScaler para ajustar os atributos numéricos a média zero e desvio padrão um.

    - O scaler foi salvo em .pkl para aplicação idêntica no conjunto de teste.

7. Reconstrução final dos datasets

    - As colunas customerID e Churn foram reincorporadas aos datasets processados.

    - Assim foram obtidos os conjuntos abt_train e abt_test, totalmente numéricos, consistentes e prontos para a modelagem.



Esse pipeline garante que todas as etapas do pré-processamento sejam reprodutíveis, rastreadas e aplicáveis de forma consistente tanto em treino quanto em teste, além de preparar a base de dados para diferentes algoritmos de machine learning.

**1. Preparação do Ambiente e Leitura dos Dados**

In [ ]:
#importar as bibliotecas
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pickle



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


O dataset de churn da Telecom possui pouco mais de 7.000 clientes e 21 variáveis, englobando características demográficas, serviços contratados e informações de faturamento. A variável-alvo é Churn (sim/não), que indica se o cliente deixou a empresa no último mês

In [ ]:
#leitura dos dados
data_path = "https://raw.githubusercontent.com/carlosfab/dsnp2/master/datasets/WA_Fn-UseC_-Telco-Customer-Churn.csv"

df00 = pd.read_csv(data_path)

print('A dimensão do dataset total é: {}'.format(df00.shape))

A dimensão do dataset total é: (7043, 21)


In [ ]:
#visualização do dataset
df00.head()

customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
0  No phone service             DSL             No  ...               No   
1                No             DSL            Yes  ...              Yes   
2                No             DSL            Yes  ...               No   
3  No phone service             DSL            Yes  ...              Yes   
4                No     Fiber optic             No  ...               No   

  TechSupport StreamingTV StreamingMovies        Contract PaperlessBilling  \
0          No          No              No  Month-to-month              Yes   
1          No          No              No        One year               No   
2          No          No              No  Month-to-month              Yes   
3         Yes          No              No        One year               No   
4          No          No              No  Month-to-month              Yes   

               PaymentMethod MonthlyCharges  TotalCharges Churn  
0           Electronic check          29.85         29.85    No  
1               Mailed check          56.95        1889.5    No  
2               Mailed check          53.85        108.15   Yes  
3  Bank transfer (automatic)          42.30       1840.75    No  
4           Electronic check          70.70        151.65   Yes  

[5 rows x 21 columns]

A coluna Churn foi convertida de valores categóricos (“Yes” e “No”) para valores numéricos binários, em que o valor 1 representa clientes que cancelaram o serviço e o valor 0 representa clientes que permaneceram ativos. Essa transformação é fundamental para viabilizar análises estatísticas e a construção de modelos preditivos, uma vez que a maioria dos algoritmos de machine learning opera de forma mais eficiente com variáveis numéricas. Além disso, essa conversão possibilita o cálculo de métricas, correlações e probabilidades associadas ao comportamento de evasão, tornando o dado mais adequado para a etapa de modelagem.

In [ ]:
#converte coluna Churn para valores binários
df00.replace({'Churn': {'Yes': 1, 'No': 0}}, inplace=True)

df00.head()


/tmp/ipython-input-2385632929.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df00.replace({'Churn': {'Yes': 1, 'No': 0}}, inplace=True)


customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
0  No phone service             DSL             No  ...               No   
1                No             DSL            Yes  ...              Yes   
2                No             DSL            Yes  ...               No   
3  No phone service             DSL            Yes  ...              Yes   
4                No     Fiber optic             No  ...               No   

  TechSupport StreamingTV StreamingMovies        Contract PaperlessBilling  \
0          No          No              No  Month-to-month              Yes   
1          No          No              No        One year               No   
2          No          No              No  Month-to-month              Yes   
3         Yes          No              No        One year               No   
4          No          No              No  Month-to-month              Yes   

               PaymentMethod MonthlyCharges  TotalCharges Churn  
0           Electronic check          29.85         29.85     0  
1               Mailed check          56.95        1889.5     0  
2               Mailed check          53.85        108.15     1  
3  Bank transfer (automatic)          42.30       1840.75     0  
4           Electronic check          70.70        151.65     1  

[5 rows x 21 columns]

**2. Validação Cruzada Holdout 70/30**

Na etapa de Validação Cruzada Holdout 70/30, o conjunto de dados foi dividido em dois subconjuntos por meio da função train_test_split da biblioteca scikit-learn, sendo 70% dos registros destinados ao treinamento do modelo e 30% reservados para teste. Essa estratégia garante que o modelo seja ajustado em uma parte da base e avaliado em outra que não foi utilizada no treinamento, possibilitando medir sua capacidade de generalização.

A escolha por essa divisão reduz o risco de overfitting, já que o desempenho é testado em dados inéditos. Além disso, definiu-se um valor fixo para o parâmetro random_state, assegurando a reprodutibilidade dos experimentos e a consistência dos resultados em diferentes execuções.

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df00, test_size=0.3, random_state=42)

train.shape, test.shape


((4930, 21), (2113, 21))

**3. Cópia do DataFrame**

Para preservar a integridade do conjunto de treino original, foi criada uma cópia do DataFrame denominada df_train_01. Essa prática é relevante porque permite aplicar transformações e realizar testes diretamente na nova estrutura sem comprometer os dados originais.

Dessa forma, garante-se maior segurança durante o processo de pré-processamento e mantém-se a possibilidade de comparar diferentes versões do mesmo conjunto de dados ao longo do desenvolvimento do projeto, facilitando análises e ajustes sem risco de perda de informações.

In [ ]:
df_train_01 = train.copy()

A função generate_metadata foi criada para auxiliar na análise exploratória do dataset, oferecendo uma visão consolidada das características principais de cada variável. Ela retorna um DataFrame com metadados que incluem o nome da coluna, tipo de dado, quantidade e percentual de valores ausentes e cardinalidade (número de valores únicos).

Essas informações são fundamentais para orientar as etapas de pré-processamento, como tratamento de nulos, codificação de variáveis categóricas e seleção de atributos. Ao ordenar as colunas pelo percentual de valores ausentes, a função também facilita a priorização das variáveis que podem demandar maior atenção durante a preparação dos dados para modelagem.

In [ ]:
def generate_metadata(dataframe):

    metadata = pd.DataFrame({
        'FEATURE': dataframe.columns,
        'TIPO_FEATURE': dataframe.dtypes,
        'QT_NULOS': dataframe.isnull().sum(),
        'PC_NULOS': round((dataframe.isnull().sum() / len(dataframe))* 100,2),
        'CARDINALIDADE': dataframe.nunique(),
    })
    metadata = metadata.sort_values(by='PC_NULOS')
    metadata = metadata.reset_index(drop=True)

    return metadata


In [ ]:
metadados = generate_metadata(df_train_01)
metadados

FEATURE TIPO_FEATURE  QT_NULOS  PC_NULOS  CARDINALIDADE
0         customerID       object         0       0.0           4930
1             gender       object         0       0.0              2
2      SeniorCitizen        int64         0       0.0              2
3            Partner       object         0       0.0              2
4         Dependents       object         0       0.0              2
5             tenure        int64         0       0.0             73
6       PhoneService       object         0       0.0              2
7      MultipleLines       object         0       0.0              3
8    InternetService       object         0       0.0              3
9     OnlineSecurity       object         0       0.0              3
10      OnlineBackup       object         0       0.0              3
11  DeviceProtection       object         0       0.0              3
12       TechSupport       object         0       0.0              3
13       StreamingTV       object         0       0.0              3
14   StreamingMovies       object         0       0.0              3
15          Contract       object         0       0.0              3
16  PaperlessBilling       object         0       0.0              2
17     PaymentMethod       object         0       0.0              4
18    MonthlyCharges      float64         0       0.0           1441
19      TotalCharges       object         0       0.0           4660
20             Churn        int64         0       0.0              2

Durante a análise exploratória, observou-se que a variável TotalCharges foi carregada como tipo object em vez de float64, devido à presença de espaços em branco no dataset. Para corrigir esse problema, os espaços foram substituídos por valores nulos (pd.NA), permitindo posteriormente a conversão adequada da coluna para formato numérico. Essa etapa é essencial, pois TotalCharges representa valores monetários acumulados e precisa estar em formato contínuo para ser utilizada corretamente em análises estatísticas e na modelagem preditiva.

In [ ]:
# Alterar "TotalCharges" para float64
df_train_01['TotalCharges'] = df_train_01['TotalCharges'].replace(' ', pd.NA)
test['TotalCharges'] = test['TotalCharges'].replace(' ', pd.NA)


In [ ]:
metadados = generate_metadata(df_train_01)
metadados


FEATURE TIPO_FEATURE  QT_NULOS  PC_NULOS  CARDINALIDADE
0         customerID       object         0      0.00           4930
1             gender       object         0      0.00              2
2      SeniorCitizen        int64         0      0.00              2
3            Partner       object         0      0.00              2
4         Dependents       object         0      0.00              2
5             tenure        int64         0      0.00             73
6       PhoneService       object         0      0.00              2
7      MultipleLines       object         0      0.00              3
8    InternetService       object         0      0.00              3
9     OnlineSecurity       object         0      0.00              3
10      OnlineBackup       object         0      0.00              3
11  DeviceProtection       object         0      0.00              3
12       TechSupport       object         0      0.00              3
13       StreamingTV       object         0      0.00              3
14   StreamingMovies       object         0      0.00              3
15          Contract       object         0      0.00              3
16  PaperlessBilling       object         0      0.00              2
17     PaymentMethod       object         0      0.00              4
18    MonthlyCharges      float64         0      0.00           1441
19             Churn        int64         0      0.00              2
20      TotalCharges       object         8      0.16           4659

In [ ]:
metadados = generate_metadata(test)
metadados

FEATURE TIPO_FEATURE  QT_NULOS  PC_NULOS  CARDINALIDADE
0         customerID       object         0      0.00           2113
1             gender       object         0      0.00              2
2      SeniorCitizen        int64         0      0.00              2
3            Partner       object         0      0.00              2
4         Dependents       object         0      0.00              2
5             tenure        int64         0      0.00             73
6       PhoneService       object         0      0.00              2
7      MultipleLines       object         0      0.00              3
8    InternetService       object         0      0.00              3
9     OnlineSecurity       object         0      0.00              3
10      OnlineBackup       object         0      0.00              3
11  DeviceProtection       object         0      0.00              3
12       TechSupport       object         0      0.00              3
13       StreamingTV       object         0      0.00              3
14   StreamingMovies       object         0      0.00              3
15          Contract       object         0      0.00              3
16  PaperlessBilling       object         0      0.00              2
17     PaymentMethod       object         0      0.00              4
18    MonthlyCharges      float64         0      0.00           1013
19             Churn        int64         0      0.00              2
20      TotalCharges       object         3      0.14           2049

**4. Substituição dos Nulos pela moda para variáveis categóricas**

Na etapa de Substituição dos Nulos pela Moda para Variáveis Categóricas, foi criada a função preencher_nulos_com_moda, que identifica automaticamente quais colunas categóricas apresentam valores ausentes e aplica a moda de cada coluna como critério de preenchimento. Essa abordagem é adequada para variáveis qualitativas, pois mantém a consistência dos dados e evita distorções que poderiam ocorrer com outras técnicas de imputação. Além disso, a função registra em um dicionário as modas utilizadas em cada coluna, garantindo transparência no processo de tratamento dos nulos e possibilitando a rastreabilidade das substituições realizadas. Essa prática contribui para preservar a representatividade dos dados e minimizar impactos negativos na etapa de modelagem.

In [ ]:
def preencher_nulos_com_moda(dataframe):
    # Obtém as colunas categóricas com valores nulos
    colunas_categoricas_nulas = dataframe.select_dtypes(include='object').columns[dataframe.select_dtypes(include='object').isnull().any()].tolist()

    # Dicionário para armazenar as modas utilizadas
    modas_utilizadas = {}

    # Preenche os valores nulos com a moda e registra a moda utilizada
    for coluna in colunas_categoricas_nulas:
        moda_coluna = dataframe[coluna].mode().iloc[0]  # Pode haver múltiplas modas, pegamos a primeira
        dataframe[coluna].fillna(moda_coluna, inplace=True)
        modas_utilizadas[coluna] = moda_coluna

    return dataframe, modas_utilizadas

Nesta etapa foi criada uma nova versão do conjunto de treino, chamada df_train_02, já com os valores nulos das variáveis categóricas preenchidos pela moda de cada coluna. Paralelamente, o dicionário modes armazena quais foram as modas utilizadas em cada variável durante o processo, permitindo documentar e rastrear as substituições realizadas. Essa estratégia garante que o conjunto de dados esteja livre de inconsistências antes da etapa de modelagem, ao mesmo tempo em que preserva a integridade do DataFrame original (df_train_01).

In [ ]:
#Preenchimento de nulos e salvamento das modas
df_train_02, modes = preencher_nulos_com_moda(df_train_01)


with open('/content/drive/MyDrive/Colab Notebooks/Projetos/Predicao de Churn - Empresa Telecom/01 - Data Prep/prd_fillna.pkl', 'wb') as f:
  pickle.dump(modes, f)

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/Projetos/Predicao de Churn - Empresa Telecom/01 - Data Prep/prd_fillna.pkl', 'rb') as f:
  loaded_modes = pickle.load(f)
loaded_modes



{}

A função fill_nulls_with_loaded_modes foi criada para aplicar nos dados novos (por exemplo, o conjunto de teste) o mesmo critério de imputação utilizado no conjunto de treino.

In [ ]:
def fill_nulls_with_loaded_modes(dataframe, loaded_modes):
    # Obtém as colunas categóricas com valores nulos
    categorical_columns_with_nulls = dataframe.select_dtypes(include='object').columns[dataframe.select_dtypes(include='object').isnull().any()].tolist()

    # Preenche os valores nulos com os modos carregados
    for column in categorical_columns_with_nulls:
        if column in loaded_modes:
            mode_column = loaded_modes[column]
            dataframe[column].fillna(mode_column, inplace=True)

    return dataframe


In [ ]:
# Imputação de Nulos no Teste com Modas Carregadas
test = fill_nulls_with_loaded_modes(test,loaded_modes)
test.shape


(2113, 21)

In [ ]:
test.head()


customerID  gender  SeniorCitizen Partner Dependents  tenure  \
185   1024-GUALD  Female              0     Yes         No       1   
2715  0484-JPBRU    Male              0      No         No      41   
3825  3620-EHIMZ  Female              0     Yes        Yes      52   
1807  6910-HADCM  Female              0      No         No       1   
132   8587-XYZSF    Male              0      No         No      67   

     PhoneService     MultipleLines InternetService       OnlineSecurity  ...  \
185            No  No phone service             DSL                   No  ...   
2715          Yes               Yes              No  No internet service  ...   
3825          Yes                No              No  No internet service  ...   
1807          Yes                No     Fiber optic                   No  ...   
132           Yes                No             DSL                   No  ...   

         DeviceProtection          TechSupport          StreamingTV  \
185                    No                   No                   No   
2715  No internet service  No internet service  No internet service   
3825  No internet service  No internet service  No internet service   
1807                  Yes                   No                   No   
132                    No                  Yes                   No   

          StreamingMovies        Contract PaperlessBilling  \
185                    No  Month-to-month              Yes   
2715  No internet service  Month-to-month              Yes   
3825  No internet service        Two year               No   
1807                   No  Month-to-month               No   
132                    No        Two year               No   

                  PaymentMethod MonthlyCharges  TotalCharges Churn  
185            Electronic check          24.80          24.8     1  
2715  Bank transfer (automatic)          25.25        996.45     0  
3825               Mailed check          19.35        1031.7     0  
1807           Electronic check          76.35         76.35     1  
132   Bank transfer (automatic)          50.55        3260.1     0  

[5 rows x 21 columns]

**5. Transformação variável TotalCharge em float64**

Esse comando converte a coluna TotalCharges do conjunto de teste para o tipo numérico, utilizando a função pd.to_numeric com o parâmetro errors="coerce". Dessa forma, qualquer valor que não puder ser convertido em número (por exemplo, strings ou espaços em branco) será transformado em NaN.

In [ ]:
df_train_02['TotalCharges'] = pd.to_numeric(df_train_02['TotalCharges'], errors="coerce")
test['TotalCharges'] = pd.to_numeric(test['TotalCharges'], errors="coerce")


**6. Eliminação de variáveis com mais 70% nulos**

Esse trecho define um ponto de corte de 70% de valores nulos para identificar variáveis que não devem ser mantidas no dataset, pois apresentam um percentual muito alto de dados ausentes. A partir da análise do DataFrame metadados, foram selecionadas todas as colunas com PC_NULOS ≥ 70, que foram armazenadas na lista lista_drop_vars. Em seguida, essas variáveis foram removidas do conjunto de treino, resultando no novo DataFrame df_train_03.

O uso desse critério tem como objetivo reduzir ruído e evitar distorções durante a modelagem, já que variáveis com grande quantidade de nulos tendem a ter baixa utilidade preditiva e podem comprometer a qualidade do modelo. A chamada final df_train_03.shape confirma as dimensões do dataset após a exclusão das colunas.

In [ ]:
missing_cutoff = 70

drop_vars_nulos = metadados[(metadados['PC_NULOS'] >= missing_cutoff)]
lista_drop_vars = list(drop_vars_nulos.FEATURE.values)

print('Variáveis que serão excluídas por alto percentual de nulos: ',lista_drop_vars)
# retirando lista de variáveis com alto percentual de nulos
df_train_03 = df_train_02.drop(axis=1,columns=lista_drop_vars)
df_train_03.shape


Variáveis que serão excluídas por alto percentual de nulos:  []


(4930, 21)

In [ ]:
# Salvar a lista em um arquivo .pkl
with open('/content/drive/MyDrive/Colab Notebooks/Projetos/Predicao de Churn - Empresa Telecom/01 - Data Prep/prd_drop_nullvars.pkl', 'wb') as f:
    pickle.dump(lista_drop_vars, f)


In [ ]:
df_train_03.head()


customerID  gender  SeniorCitizen Partner Dependents  tenure  \
1695  4906-ZHGPK    Male              0     Yes        Yes      54   
1095  7439-DKZTW    Male              0      No         No       1   
3889  2592-HODOV    Male              0      No         No      13   
3667  7826-VVKWT  Female              1     Yes        Yes      24   
2902  1658-TJVOA  Female              1      No         No       6   

     PhoneService MultipleLines InternetService       OnlineSecurity  ...  \
1695          Yes           Yes             DSL                   No  ...   
1095          Yes            No     Fiber optic                   No  ...   
3889          Yes            No              No  No internet service  ...   
3667          Yes            No     Fiber optic                   No  ...   
2902          Yes            No     Fiber optic                  Yes  ...   

         DeviceProtection          TechSupport          StreamingTV  \
1695                   No                  Yes                  Yes   
1095                   No                   No                  Yes   
3889  No internet service  No internet service  No internet service   
3667                  Yes                   No                  Yes   
2902                   No                   No                   No   

          StreamingMovies        Contract PaperlessBilling  \
1695                   No        One year              Yes   
1095                   No  Month-to-month              Yes   
3889  No internet service        One year               No   
3667                  Yes        Two year              Yes   
2902                   No  Month-to-month              Yes   

                PaymentMethod MonthlyCharges  TotalCharges  Churn  
1695         Electronic check          70.70       3770.00      0  
1095         Electronic check          80.55         80.55      0  
3889  Credit card (automatic)          19.30        259.65      0  
3667         Electronic check          96.55       2263.45      0  
2902         Electronic check          74.10        450.90      0  

[5 rows x 21 columns]

**7. Exclusão Unnamed:0, customerID e Churn do Tratamento de nulos**

Esse comando removeu as colunas 'Unnamed: 0', 'customerID' e 'Churn' do DataFrame antes do tratamento de nulos, resultando no novo conjunto df_train_04. A exclusão dessas variáveis tem justificativa prática:
  
  - Unnamed: 0' é apenas um índice criado na importação do dataset e não possui relevância analítica.

  - customerID' é um identificador único, que não agrega valor preditivo e poderia apenas introduzir ruído no modelo.

  - 'Churn' é a variável alvo, portanto deve ser mantida separada para não sofrer transformações indevidas durante o pré-processamento.

A criação do DataFrame df_train_04 garante que apenas as variáveis realmente explicativas sejam consideradas nas etapas seguintes de tratamento e modelagem.

In [ ]:
# Retirar 'Unnamed: 0', 'customerID' e 'Churn' do tratamento de nulos
df_train_04 = df_train_03.drop(columns=['Unnamed: 0', 'customerID', 'Churn'], errors='ignore')
df_train_04.head()



gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
1695    Male              0     Yes        Yes      54          Yes   
1095    Male              0      No         No       1          Yes   
3889    Male              0      No         No      13          Yes   
3667  Female              1     Yes        Yes      24          Yes   
2902  Female              1      No         No       6          Yes   

     MultipleLines InternetService       OnlineSecurity         OnlineBackup  \
1695           Yes             DSL                   No                  Yes   
1095            No     Fiber optic                   No                   No   
3889            No              No  No internet service  No internet service   
3667            No     Fiber optic                   No                   No   
2902            No     Fiber optic                  Yes                   No   

         DeviceProtection          TechSupport          StreamingTV  \
1695                   No                  Yes                  Yes   
1095                   No                   No                  Yes   
3889  No internet service  No internet service  No internet service   
3667                  Yes                   No                  Yes   
2902                   No                   No                   No   

          StreamingMovies        Contract PaperlessBilling  \
1695                   No        One year              Yes   
1095                   No  Month-to-month              Yes   
3889  No internet service        One year               No   
3667                  Yes        Two year              Yes   
2902                   No  Month-to-month              Yes   

                PaymentMethod  MonthlyCharges  TotalCharges  
1695         Electronic check           70.70       3770.00  
1095         Electronic check           80.55         80.55  
3889  Credit card (automatic)           19.30        259.65  
3667         Electronic check           96.55       2263.45  
2902         Electronic check           74.10        450.90

In [ ]:
metadados = generate_metadata(df_train_04)


**8. Tratamento de categóricos de alta cardinalidade (LabelEncoder)**

Na etapa de Tratamento de Categóricos de Alta Cardinalidade, foi utilizado o método Label Encoding para converter variáveis categóricas em valores numéricos inteiros. Esse procedimento é especialmente indicado para atributos com grande número de categorias, nos quais técnicas como One-Hot Encoding poderiam gerar um aumento excessivo de dimensionalidade e prejudicar a eficiência do modelo.

O Label Encoder atribui um código numérico único a cada categoria dentro de uma variável, mantendo a informação de forma compacta e preservando a consistência do dataset. Apesar de simples, esse método deve ser aplicado com cautela, já que cria uma ordenação artificial entre as categorias; no entanto, para modelos baseados em árvores, como Random Forest e Gradient Boosting, esse efeito não gera impacto negativo relevante.

In [ ]:
card_cutoff = 20
df_categ_labelenc = metadados[(metadados['CARDINALIDADE'] > card_cutoff) & (metadados['TIPO_FEATURE'] == 'object')]
lista_vars_abt = list(df_train_03.columns)
lista_lenc = list(df_categ_labelenc.FEATURE.values)

for item in lista_drop_vars:
    if item in lista_lenc:
        lista_lenc.remove(item)

print('Lista de vars para Label Encoding: ',lista_lenc)


Lista de vars para Label Encoding:  []


In [ ]:
df_train_04.head()

gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
1695    Male              0     Yes        Yes      54          Yes   
1095    Male              0      No         No       1          Yes   
3889    Male              0      No         No      13          Yes   
3667  Female              1     Yes        Yes      24          Yes   
2902  Female              1      No         No       6          Yes   

     MultipleLines InternetService       OnlineSecurity         OnlineBackup  \
1695           Yes             DSL                   No                  Yes   
1095            No     Fiber optic                   No                   No   
3889            No              No  No internet service  No internet service   
3667            No     Fiber optic                   No                   No   
2902            No     Fiber optic                  Yes                   No   

         DeviceProtection          TechSupport          StreamingTV  \
1695                   No                  Yes                  Yes   
1095                   No                   No                  Yes   
3889  No internet service  No internet service  No internet service   
3667                  Yes                   No                  Yes   
2902                   No                   No                   No   

          StreamingMovies        Contract PaperlessBilling  \
1695                   No        One year              Yes   
1095                   No  Month-to-month              Yes   
3889  No internet service        One year               No   
3667                  Yes        Two year              Yes   
2902                   No  Month-to-month              Yes   

                PaymentMethod  MonthlyCharges  TotalCharges  
1695         Electronic check           70.70       3770.00  
1095         Electronic check           80.55         80.55  
3889  Credit card (automatic)           19.30        259.65  
3667         Electronic check           96.55       2263.45  
2902         Electronic check           74.10        450.90

Esse trecho implementa o processo de Label Encoding para variáveis categóricas de alta cardinalidade e garante que os encoders treinados fiquem armazenados para uso futuro.

In [ ]:

import pickle
from sklearn.preprocessing import LabelEncoder

encoders = {}

for col in lista_lenc:
    encoder = LabelEncoder()
    df_train_04[col] = encoder.fit_transform(df_train_04[col])

    # Armazena o encoder para a coluna atual em um dicionário
    encoders[col] = encoder

# Salva o dicionário de encoders e a lista de colunas em um arquivo .pkl
data_to_serialize = {
    'encoders': encoders,
    'columns': lista_lenc
}


with open('/content/drive/MyDrive/Colab Notebooks/Projetos/Predicao de Churn - Empresa Telecom/01 - Data Prep/prd_labelenc.pkl', 'wb') as f:
    pickle.dump(data_to_serialize, f)

In [ ]:
test.head()


customerID  gender  SeniorCitizen Partner Dependents  tenure  \
185   1024-GUALD  Female              0     Yes         No       1   
2715  0484-JPBRU    Male              0      No         No      41   
3825  3620-EHIMZ  Female              0     Yes        Yes      52   
1807  6910-HADCM  Female              0      No         No       1   
132   8587-XYZSF    Male              0      No         No      67   

     PhoneService     MultipleLines InternetService       OnlineSecurity  ...  \
185            No  No phone service             DSL                   No  ...   
2715          Yes               Yes              No  No internet service  ...   
3825          Yes                No              No  No internet service  ...   
1807          Yes                No     Fiber optic                   No  ...   
132           Yes                No             DSL                   No  ...   

         DeviceProtection          TechSupport          StreamingTV  \
185                    No                   No                   No   
2715  No internet service  No internet service  No internet service   
3825  No internet service  No internet service  No internet service   
1807                  Yes                   No                   No   
132                    No                  Yes                   No   

          StreamingMovies        Contract PaperlessBilling  \
185                    No  Month-to-month              Yes   
2715  No internet service  Month-to-month              Yes   
3825  No internet service        Two year               No   
1807                   No  Month-to-month               No   
132                    No        Two year               No   

                  PaymentMethod MonthlyCharges  TotalCharges  Churn  
185            Electronic check          24.80         24.80      1  
2715  Bank transfer (automatic)          25.25        996.45      0  
3825               Mailed check          19.35       1031.70      0  
1807           Electronic check          76.35         76.35      1  
132   Bank transfer (automatic)          50.55       3260.10      0  

[5 rows x 21 columns]

Esse trecho garante que o mesmo Label Encoding aplicado no treino seja replicado no conjunto de teste, mantendo consistência no pré-processamento.

In [ ]:
# Carregar os encoders e a lista de colunas
with open('/content/drive/MyDrive/Colab Notebooks/Projetos/Predicao de Churn - Empresa Telecom/01 - Data Prep/prd_labelenc.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

loaded_encoders = loaded_data['encoders']
loaded_columns = loaded_data['columns']

# Suponha test_df como sua base de teste
for col in loaded_columns:
    if col in loaded_encoders:
        # Transforma a coluna usando o encoder carregado
        test[col] = loaded_encoders[col].transform(test[col])

**9. Tratamento para categóricas de baixa cardinalidade**



Nesta etapa, as variáveis categóricas de baixa cardinalidade (até 20 categorias) foram transformadas em variáveis numéricas por meio do One-Hot Encoding, que cria colunas binárias (0 ou 1) para representar cada categoria. Para evitar redundância, foi utilizada a opção drop='first', eliminando a primeira categoria de cada variável. Em seguida, as colunas originais foram removidas e substituídas pelas novas variáveis dummy, resultando no DataFrame df_train_05, agora composto apenas por atributos numéricos. Por fim, o encoder treinado e a lista de colunas transformadas foram salvos em um arquivo .pkl, garantindo que o mesmo procedimento possa ser replicado no conjunto de teste ou em futuros dados da empresa.

In [ ]:
import pickle
from sklearn.preprocessing import OneHotEncoder

card_cutoff = 20
df_categ_onehot = metadados[(metadados['CARDINALIDADE'] <= card_cutoff) & (metadados['TIPO_FEATURE'] == 'object')]
lista_onehot = list(df_categ_onehot.FEATURE.values)
print('Lista de vars para OneHot Encoding: ',lista_onehot)

# Instanciando o encoder
encoder = OneHotEncoder(drop='first', sparse_output=False)

# Aplicando o one-hot encoding
encoded_data = encoder.fit_transform(df_train_04[lista_onehot])
encoded_cols = encoder.get_feature_names_out(lista_onehot)
encoded_df = pd.DataFrame(encoded_data, columns=encoded_cols, index=df_train_03.index)

df_train_05 = pd.concat([df_train_04.drop(lista_onehot, axis=1), encoded_df], axis=1)

# Salva o encoder e a lista de colunas em um arquivo .pkl
data_to_serialize = {
    'encoder': encoder,
    'columns': lista_onehot
}

with open('/content/drive/MyDrive/Colab Notebooks/Projetos/Predicao de Churn - Empresa Telecom/01 - Data Prep/prd_onehotenc.pkl', 'wb') as f:
    pickle.dump(data_to_serialize, f)

Lista de vars para OneHot Encoding:  ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']


Nessa etapa foi carregado o One-Hot Encoder salvo anteriormente e aplica exatamente a mesma transformação no conjunto de teste. Primeiro, o arquivo .pkl é aberto para recuperar o encoder treinado e a lista de colunas categóricas. Em seguida, essas colunas no test são transformadas em variáveis dummy de acordo com o padrão aprendido no treino. As colunas originais são removidas e substituídas pelas novas variáveis binárias, resultando em um conjunto de teste totalmente numérico e consistente com o dataset de treino.

In [ ]:
# Carregar o encoder e a lista de colunas
with open('/content/drive/MyDrive/Colab Notebooks/Projetos/Predicao de Churn - Empresa Telecom/01 - Data Prep/prd_onehotenc.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

loaded_encoder = loaded_data['encoder']
loaded_columns = loaded_data['columns']

# Suponha test_df como sua base de teste
encoded_data_test = loaded_encoder.transform(test[loaded_columns])
encoded_cols_test = loaded_encoder.get_feature_names_out(loaded_columns)
encoded_df_test = pd.DataFrame(encoded_data_test, columns=encoded_cols_test, index=test.index)

test = pd.concat([test.drop(loaded_columns, axis=1), encoded_df_test], axis=1)



In [ ]:
df_train_05.head()


SeniorCitizen  tenure  MonthlyCharges  TotalCharges  gender_Male  \
1695              0      54           70.70       3770.00          1.0   
1095              0       1           80.55         80.55          1.0   
3889              0      13           19.30        259.65          1.0   
3667              1      24           96.55       2263.45          0.0   
2902              1       6           74.10        450.90          0.0   

      Partner_Yes  Dependents_Yes  PhoneService_Yes  \
1695          1.0             1.0               1.0   
1095          0.0             0.0               1.0   
3889          0.0             0.0               1.0   
3667          1.0             1.0               1.0   
2902          0.0             0.0               1.0   

      MultipleLines_No phone service  MultipleLines_Yes  ...  \
1695                             0.0                1.0  ...   
1095                             0.0                0.0  ...   
3889                             0.0                0.0  ...   
3667                             0.0                0.0  ...   
2902                             0.0                0.0  ...   

      StreamingTV_No internet service  StreamingTV_Yes  \
1695                              0.0              1.0   
1095                              0.0              1.0   
3889                              1.0              0.0   
3667                              0.0              1.0   
2902                              0.0              0.0   

      StreamingMovies_No internet service  StreamingMovies_Yes  \
1695                                  0.0                  0.0   
1095                                  0.0                  0.0   
3889                                  1.0                  0.0   
3667                                  0.0                  1.0   
2902                                  0.0                  0.0   

      Contract_One year  Contract_Two year  PaperlessBilling_Yes  \
1695                1.0                0.0                   1.0   
1095                0.0                0.0                   1.0   
3889                1.0                0.0                   0.0   
3667                0.0                1.0                   1.0   
2902                0.0                0.0                   1.0   

      PaymentMethod_Credit card (automatic)  PaymentMethod_Electronic check  \
1695                                    0.0                             1.0   
1095                                    0.0                             1.0   
3889                                    1.0                             0.0   
3667                                    0.0                             1.0   
2902                                    0.0                             1.0   

      PaymentMethod_Mailed check  
1695                         0.0  
1095                         0.0  
3889                         0.0  
3667                         0.0  
2902                         0.0  

[5 rows x 30 columns]

**10. Aplicar normalização a toda tabela de modelagem tratada ate este ponto**


Esse trecho aplica a padronização dos dados numéricos utilizando o StandardScaler, que transforma as variáveis para média zero e desvio padrão igual a um. As colunas irrelevantes CustomerID, Churn e Unnamed: 0 foram excluídas do processo, já que não devem ser escalonadas. O scaler foi ajustado e aplicado sobre o conjunto de treino, resultando em um novo DataFrame padronizado (df_train_06). Por fim, o objeto scaler foi salvo em um arquivo .pkl, garantindo que a mesma transformação possa ser reproduzida no conjunto de teste e em dados futuros.

In [ ]:
import pickle
from sklearn.preprocessing import StandardScaler

# Excluindo IDs e Targets
df_id_target = metadados[(metadados['FEATURE'] == 'CustomerID') | (metadados['FEATURE'] == 'Churn') | (metadados['FEATURE'] == 'Unnamed: 0')]
lista_id_target = list(df_id_target.FEATURE.values)
print('Lista de CustomerID, Unnamed: 0 e Churn: ',lista_id_target)

# Instanciando o scaler
scaler = StandardScaler()

# Padronizando a base de treino
df_train_05_scaled = scaler.fit_transform(df_train_05)
df_train_06 = pd.DataFrame(df_train_05_scaled, columns=df_train_05.columns, index=df_train_05.index)

# Salva o scaler em um arquivo .pkl
with open('/content/drive/MyDrive/Colab Notebooks/Projetos/Predicao de Churn - Empresa Telecom/01 - Data Prep/prd_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)



Lista de CustomerID, Unnamed: 0 e Churn:  []


In [ ]:
df_train_06.head()

SeniorCitizen    tenure  MonthlyCharges  TotalCharges  gender_Male  \
1695      -0.436831  0.881078        0.195927      0.654499     0.968056   
1095      -0.436831 -1.284263        0.522755     -0.974070     0.968056   
3889      -0.436831 -0.793997       -1.509551     -0.895013     0.968056   
3667       2.289215 -0.344587        1.053643     -0.010511    -1.032999   
2902       2.289215 -1.079985        0.308740     -0.810593    -1.032999   

      Partner_Yes  Dependents_Yes  PhoneService_Yes  \
1695     1.030066        1.518718          0.323478   
1095    -0.970811       -0.658450          0.323478   
3889    -0.970811       -0.658450          0.323478   
3667     1.030066        1.518718          0.323478   
2902    -0.970811       -0.658450          0.323478   

      MultipleLines_No phone service  MultipleLines_Yes  ...  \
1695                       -0.323478           1.153199  ...   
1095                       -0.323478          -0.867153  ...   
3889                       -0.323478          -0.867153  ...   
3667                       -0.323478          -0.867153  ...   
2902                       -0.323478          -0.867153  ...   

      StreamingTV_No internet service  StreamingTV_Yes  \
1695                        -0.528699         1.259591   
1095                        -0.528699         1.259591   
3889                         1.891436        -0.793909   
3667                        -0.528699         1.259591   
2902                        -0.528699        -0.793909   

      StreamingMovies_No internet service  StreamingMovies_Yes  \
1695                            -0.528699            -0.800374   
1095                            -0.528699            -0.800374   
3889                             1.891436            -0.800374   
3667                            -0.528699             1.249415   
2902                            -0.528699            -0.800374   

      Contract_One year  Contract_Two year  PaperlessBilling_Yes  \
1695           1.917660          -0.562826              0.841222   
1095          -0.521469          -0.562826              0.841222   
3889           1.917660          -0.562826             -1.188746   
3667          -0.521469           1.776747              0.841222   
2902          -0.521469          -0.562826              0.841222   

      PaymentMethod_Credit card (automatic)  PaymentMethod_Electronic check  \
1695                              -0.524928                        1.399708   
1095                              -0.524928                        1.399708   
3889                               1.905022                       -0.714435   
3667                              -0.524928                        1.399708   
2902                              -0.524928                        1.399708   

      PaymentMethod_Mailed check  
1695                   -0.539991  
1095                   -0.539991  
3889                   -0.539991  
3667                   -0.539991  
2902                   -0.539991  

[5 rows x 30 columns]

Esse código garante que o conjunto de teste seja padronizado com o mesmo scaler utilizado no treino, mantendo consistência no pré-processamento.

Primeiro, as colunas customerID, Churn e Unnamed: 0 são removidas do DataFrame de teste, pois não devem ser escalonadas. Em seguida, o arquivo .pkl contendo o StandardScaler treinado no conjunto de treino é carregado. Esse scaler é então aplicado ao df_test_aux, transformando as variáveis numéricas para média zero e desvio padrão igual a um, de acordo com os parâmetros aprendidos no treino. Por fim, é criado o DataFrame test_df, já padronizado, preservando os nomes das colunas e os índices originais.

Assim, garante-se que treino e teste estejam na mesma escala, condição essencial para a correta avaliação e generalização do modelo.

In [ ]:

# Colunas que foram retiradas da tabela:
list_columns_drop = ['customerID','Churn','Unnamed: 0']
df_test_aux = test.drop(axis=1,columns=list_columns_drop, errors='ignore')



# Carregar o scaler
with open('/content/drive/MyDrive/Colab Notebooks/Projetos/Predicao de Churn - Empresa Telecom/01 - Data Prep/prd_scaler.pkl', 'rb') as f:
    loaded_scaler = pickle.load(f)

# Suponha test_df como sua base de teste
test_df_scaled = loaded_scaler.transform(df_test_aux)
test_df = pd.DataFrame(test_df_scaled, columns=df_test_aux.columns, index=df_test_aux.index)
test_df.head()

SeniorCitizen    tenure  MonthlyCharges  TotalCharges  gender_Male  \
185       -0.436831 -1.284263       -1.327058     -0.998679    -1.032999   
2715      -0.436831  0.349957       -1.312127     -0.569780     0.968056   
3825      -0.436831  0.799367       -1.507892     -0.554221    -1.032999   
1807      -0.436831 -1.284263        0.383397     -0.975924    -1.032999   
132       -0.436831  1.412199       -0.472660      0.429422     0.968056   

      Partner_Yes  Dependents_Yes  PhoneService_Yes  \
185      1.030066       -0.658450         -3.091399   
2715    -0.970811       -0.658450          0.323478   
3825     1.030066        1.518718          0.323478   
1807    -0.970811       -0.658450          0.323478   
132     -0.970811       -0.658450          0.323478   

      MultipleLines_No phone service  MultipleLines_Yes  ...  \
185                         3.091399          -0.867153  ...   
2715                       -0.323478           1.153199  ...   
3825                       -0.323478          -0.867153  ...   
1807                       -0.323478          -0.867153  ...   
132                        -0.323478          -0.867153  ...   

      StreamingTV_No internet service  StreamingTV_Yes  \
185                         -0.528699        -0.793909   
2715                         1.891436        -0.793909   
3825                         1.891436        -0.793909   
1807                        -0.528699        -0.793909   
132                         -0.528699        -0.793909   

      StreamingMovies_No internet service  StreamingMovies_Yes  \
185                             -0.528699            -0.800374   
2715                             1.891436            -0.800374   
3825                             1.891436            -0.800374   
1807                            -0.528699            -0.800374   
132                             -0.528699            -0.800374   

      Contract_One year  Contract_Two year  PaperlessBilling_Yes  \
185           -0.521469          -0.562826              0.841222   
2715          -0.521469          -0.562826              0.841222   
3825          -0.521469           1.776747             -1.188746   
1807          -0.521469          -0.562826             -1.188746   
132           -0.521469           1.776747             -1.188746   

      PaymentMethod_Credit card (automatic)  PaymentMethod_Electronic check  \
185                               -0.524928                        1.399708   
2715                              -0.524928                       -0.714435   
3825                              -0.524928                       -0.714435   
1807                              -0.524928                        1.399708   
132                               -0.524928                       -0.714435   

      PaymentMethod_Mailed check  
185                    -0.539991  
2715                   -0.539991  
3825                    1.851883  
1807                   -0.539991  
132                    -0.539991  

[5 rows x 30 columns]

Esse trecho recompõe os datasets finais após o pré-processamento. Como durante a etapa de transformação as colunas customerID e Churn haviam sido retiradas (por serem identificador e variável alvo, respectivamente), agora elas são trazidas de volta para os conjuntos já tratados.

No abt_train, o DataFrame padronizado (df_train_06) é combinado com as colunas originais customerID e Churn do conjunto de treino, preservando a correspondência pelas posições dos índices.

Da mesma forma, no abt_test, o DataFrame padronizado de teste (test_df) é unido às mesmas colunas do conjunto de teste.

O resultado são as tabelas abt_train e abt_test, completas, padronizadas e contendo novamente o identificador e a variável alvo, prontas para a etapa de modelagem.

In [ ]:
# Trazer CustomerID e Churn de volta para a tabela pós dataprep
abt_train = df_train_06.merge(train[['customerID', 'Churn']],
                              left_index=True, right_index=True, how='inner')

abt_test = test_df.merge(test[['customerID', 'Churn']],
                         left_index=True, right_index=True, how='inner')



In [ ]:
abt_train.head()


SeniorCitizen    tenure  MonthlyCharges  TotalCharges  gender_Male  \
1695      -0.436831  0.881078        0.195927      0.654499     0.968056   
1095      -0.436831 -1.284263        0.522755     -0.974070     0.968056   
3889      -0.436831 -0.793997       -1.509551     -0.895013     0.968056   
3667       2.289215 -0.344587        1.053643     -0.010511    -1.032999   
2902       2.289215 -1.079985        0.308740     -0.810593    -1.032999   

      Partner_Yes  Dependents_Yes  PhoneService_Yes  \
1695     1.030066        1.518718          0.323478   
1095    -0.970811       -0.658450          0.323478   
3889    -0.970811       -0.658450          0.323478   
3667     1.030066        1.518718          0.323478   
2902    -0.970811       -0.658450          0.323478   

      MultipleLines_No phone service  MultipleLines_Yes  ...  \
1695                       -0.323478           1.153199  ...   
1095                       -0.323478          -0.867153  ...   
3889                       -0.323478          -0.867153  ...   
3667                       -0.323478          -0.867153  ...   
2902                       -0.323478          -0.867153  ...   

      StreamingMovies_No internet service  StreamingMovies_Yes  \
1695                            -0.528699            -0.800374   
1095                            -0.528699            -0.800374   
3889                             1.891436            -0.800374   
3667                            -0.528699             1.249415   
2902                            -0.528699            -0.800374   

      Contract_One year  Contract_Two year  PaperlessBilling_Yes  \
1695           1.917660          -0.562826              0.841222   
1095          -0.521469          -0.562826              0.841222   
3889           1.917660          -0.562826             -1.188746   
3667          -0.521469           1.776747              0.841222   
2902          -0.521469          -0.562826              0.841222   

      PaymentMethod_Credit card (automatic)  PaymentMethod_Electronic check  \
1695                              -0.524928                        1.399708   
1095                              -0.524928                        1.399708   
3889                               1.905022                       -0.714435   
3667                              -0.524928                        1.399708   
2902                              -0.524928                        1.399708   

      PaymentMethod_Mailed check  customerID  Churn  
1695                   -0.539991  4906-ZHGPK      0  
1095                   -0.539991  7439-DKZTW      0  
3889                   -0.539991  2592-HODOV      0  
3667                   -0.539991  7826-VVKWT      0  
2902                   -0.539991  1658-TJVOA      0  

[5 rows x 32 columns]

In [ ]:
abt_test.head()


SeniorCitizen    tenure  MonthlyCharges  TotalCharges  gender_Male  \
185       -0.436831 -1.284263       -1.327058     -0.998679    -1.032999   
2715      -0.436831  0.349957       -1.312127     -0.569780     0.968056   
3825      -0.436831  0.799367       -1.507892     -0.554221    -1.032999   
1807      -0.436831 -1.284263        0.383397     -0.975924    -1.032999   
132       -0.436831  1.412199       -0.472660      0.429422     0.968056   

      Partner_Yes  Dependents_Yes  PhoneService_Yes  \
185      1.030066       -0.658450         -3.091399   
2715    -0.970811       -0.658450          0.323478   
3825     1.030066        1.518718          0.323478   
1807    -0.970811       -0.658450          0.323478   
132     -0.970811       -0.658450          0.323478   

      MultipleLines_No phone service  MultipleLines_Yes  ...  \
185                         3.091399          -0.867153  ...   
2715                       -0.323478           1.153199  ...   
3825                       -0.323478          -0.867153  ...   
1807                       -0.323478          -0.867153  ...   
132                        -0.323478          -0.867153  ...   

      StreamingMovies_No internet service  StreamingMovies_Yes  \
185                             -0.528699            -0.800374   
2715                             1.891436            -0.800374   
3825                             1.891436            -0.800374   
1807                            -0.528699            -0.800374   
132                             -0.528699            -0.800374   

      Contract_One year  Contract_Two year  PaperlessBilling_Yes  \
185           -0.521469          -0.562826              0.841222   
2715          -0.521469          -0.562826              0.841222   
3825          -0.521469           1.776747             -1.188746   
1807          -0.521469          -0.562826             -1.188746   
132           -0.521469           1.776747             -1.188746   

      PaymentMethod_Credit card (automatic)  PaymentMethod_Electronic check  \
185                               -0.524928                        1.399708   
2715                              -0.524928                       -0.714435   
3825                              -0.524928                       -0.714435   
1807                              -0.524928                        1.399708   
132                               -0.524928                       -0.714435   

      PaymentMethod_Mailed check  customerID  Churn  
185                    -0.539991  1024-GUALD      1  
2715                   -0.539991  0484-JPBRU      0  
3825                    1.851883  3620-EHIMZ      0  
1807                   -0.539991  6910-HADCM      1  
132                    -0.539991  8587-XYZSF      0  

[5 rows x 32 columns]

**11. Salvando tabelas de treino e teste pós preparação dos dados**

In [ ]:
abt_train.to_csv('/content/drive/MyDrive/Colab Notebooks/Projetos/Predicao de Churn - Empresa Telecom/abt_train.csv')
abt_test.to_csv('/content/drive/MyDrive/Colab Notebooks/Projetos/Predicao de Churn - Empresa Telecom/abt_test.csv')


#**Conclusão**

A etapa de preparação dos dados garantiu que o conjunto estivesse limpo, padronizado e consistente, pronto para ser utilizado em algoritmos de machine learning. Foram tratados valores nulos, removidas variáveis irrelevantes, aplicados métodos de codificação adequados para variáveis categóricas (Label Encoding e One-Hot Encoding) e realizada a padronização das variáveis numéricas. Além disso, cada transformação foi registrada e salva em arquivos .pkl, assegurando reprodutibilidade e aplicação consistente também no conjunto de teste. O resultado final são as bases abt_train e abt_test, estruturadas exclusivamente com variáveis numéricas e a variável alvo Churn, já preparadas para a modelagem.

Com os dados devidamente preparados, a próxima fase é a seleção de variáveis (Feature Selection). O objetivo é identificar os atributos mais relevantes para a previsão de churn, reduzindo redundâncias, eliminando ruído e melhorando a eficiência do modelo. Esse processo contribui para aumentar a interpretabilidade, reduzir o tempo de processamento e, em muitos casos, elevar a performance do algoritmo, já que o modelo passa a focar nas variáveis com maior poder preditivo.